# Intelligent Systems Assignment 2

## Bayes' net inference

**Names:**

Alexander Gonzalez Castañeda   -   Camilo Andrés Rodriguez

**IDs:**

1032452063 - 

In [137]:
class Directions:
    NORTH = 'North'
    SOUTH = 'South'
    EAST = 'East'
    WEST = 'West'
    STOP = 'Stop'

### a. Bayes' net for instant perception and position.

Build a Bayes' net that represent the relationships between the random variables. Based on it, write an expression for the joint probability distribution of all the variables.

$P(E_{N}, E_{S}, E_{E}, E_{W}, X)=P(E_{N})*P(E_{S}|E_{N})*P(E_{E}|E_{N},E_{S})*P(E_{W}|E_{N},E_{S},E_{E})*P(X|E_{N},E_{S},E_{E},E_{W})$

<img src="bayesNetA.png" />


In [138]:
#Joint Probabiity dictionary{ X:[N, S, E, W, P(X,N,S,E,W)]*16 }
jointProb = {(x,y):[None]*16 for x in range(1,7) for y in range(1,6)}

for table in jointProb.values():
    for j in range(0, 16):
        table[j] = [0]*5
        if j%2==1: table[j][3]=1
        if j%4>=2: table[j][2]=1
        if j%8>=4: table[j][1]=1
        if j%16>=8: table[j][0]=1
jointProb[(3,3)][0][4]=1.0/24.0
jointProb[(1,3)][1][4]=1.0/24.0
jointProb[(6,3)][2][4]=1.0/24.0
for x in [(1,4), (3,4), (6,4), (1,2), (3,2), (6,2)]:
    jointProb[x][3][4]=1.0/24.0
jointProb[(3,1)][4][4]=1.0/24.0
jointProb[(1,1)][5][4]=1.0/24.0
jointProb[(6,1)][6][4]=1.0/24.0
jointProb[(3,5)][8][4]=1.0/24.0
jointProb[(1,5)][9][4]=1.0/24.0
jointProb[(6,5)][10][4]=1.0/24.0
for x in [(2,5), (4,5), (5,5), (2,3), (4,3), (5,3), (2,1), (4,1), (5,1)]:
    jointProb[x][12][4]=1.0/24.0

### b. Probability functions calculated from the instant model.

Assuming an uniform distribution for the Pacman position probability, write functions to calculate the following probabilities:

i. $P(X=x|E_{N}=e_{N},E_{S}=e_{S})$

In [139]:

def P_1(eps, E_N, E_S):
    
    '''
    Calculates: P(X=x|E_{N}=e_{N},E_{S}=e_{S})
    Arguments: E_N, E_S \in {True,False}
               0 <= eps <= 1 (epsilon)    
    '''
    def findModifications(mod, query, result, indices):
        if mod==0:
            result.append(query)
            return [query]
        for i in range(0,len(query)):
            if i in indices: continue
            a=str(int(not int(query[i])))
            newQ=query[:i]+a+query[i+1:]
            findModifications(mod-1, newQ, result, indices+[i])
        return list(set(result))
    
    def perceptsJointProb(eps, S):    
        global jointProb
        localJointProb={}
        indices={}
        if Directions.NORTH in S: indices[0]=S[Directions.NORTH]
        if Directions.SOUTH in S: indices[1]=S[Directions.SOUTH]
        if Directions.EAST in S: indices[2]=S[Directions.EAST]
        if Directions.WEST in S: indices[3]=S[Directions.WEST]

        for x in jointProb.values():
            for row in x:
                key=''
                for index in indices.keys():
                    key=key+str(row[index])
                if key not in localJointProb: localJointProb[key]=row[4]
                else: localJointProb[key]+=row[4]

        query=''
        for i in range(0,4):
            if i not in indices: continue
            query=query+str(int(indices[i]))
        prob=0
        for mod in range (0, len(query)+1):
            for newQ in findModifications(mod, query, [], []):
                prob+=(localJointProb[newQ])*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
        return prob
    
    global jointProb
    localJointProb={}

    for x in jointProb.keys():
        localJointProb[x]={}
        for row in jointProb[x]:
            key=''
            for index in (0,1):
                key=key+str(row[index])
            if key not in localJointProb[x]: localJointProb[x][key]=row[4]
            else: localJointProb[x][key]+=row[4]
    
    query=str(int(E_N))+str(int(E_S))
    pd = {(x,y):0 for x in range(1,7) for y in range(1,6)}
    for x in localJointProb.keys():
        prob=0
        for mod in range (0, len(query)+1):
            for newQ in findModifications(mod, query, [], []):
                prob+=(localJointProb[x][newQ])*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
        pd[x]=prob/perceptsJointProb(eps, {Directions.NORTH: E_N, Directions.SOUTH: E_S})
    return pd

P_1(0.0, True, False)

{(1, 1): 0.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (1, 5): 0.3333333333333333,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (2, 5): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (3, 5): 0.3333333333333333,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (4, 5): 0.0,
 (5, 1): 0.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (5, 5): 0.0,
 (6, 1): 0.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (6, 5): 0.3333333333333333}

ii. $P(E_{E}=e_{E}|E_{N}=e_{N},E_{S}=E_{S})$

In [140]:
def P_2(eps, E_N, E_S):
    '''
    Calculates: P(E_{E}=e_{E}|E_{N}=e_{N},E_{S}=E_{S})
    Arguments: E_N, E_S \in {True,False}
               0 <= eps <= 1
    '''
    def findModifications(mod, query, result, indices):
        if mod==0:
            result.append(query)
            return [query]
        for i in range(0,len(query)):
            if i in indices: continue
            a=str(int(not int(query[i])))
            newQ=query[:i]+a+query[i+1:]
            findModifications(mod-1, newQ, result, indices+[i])
        return list(set(result))
    
    def perceptsJointProb(eps, S):    
        global jointProb
        localJointProb={}
        indices={}
        if Directions.NORTH in S: indices[0]=S[Directions.NORTH]
        if Directions.SOUTH in S: indices[1]=S[Directions.SOUTH]
        if Directions.EAST in S: indices[2]=S[Directions.EAST]
        if Directions.WEST in S: indices[3]=S[Directions.WEST]

        for x in jointProb.values():
            for row in x:
                key=''
                for index in indices.keys():
                    key=key+str(row[index])
                if key not in localJointProb: localJointProb[key]=row[4]
                else: localJointProb[key]+=row[4]

        query=''
        for i in range(0,4):
            if i not in indices: continue
            query=query+str(int(indices[i]))
        prob=0
        for mod in range (0, len(query)+1):
            for newQ in findModifications(mod, query, [], []):
                prob+=(localJointProb[newQ])*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
        return prob
    
    pd = {True:0, False:0}
    for val in pd.keys():
        prob=perceptsJointProb(eps, {Directions.NORTH: E_N, Directions.SOUTH: E_S, Directions.EAST: val})/perceptsJointProb(eps, {Directions.NORTH: E_N, Directions.SOUTH: E_S})
        pd[val]=prob
    return pd

P_2(0.2, True, False)

{False: 0.5804878048780489, True: 0.4195121951219512}

iii. $P(S)$, where $S\subseteq\{e_{N},e_{S},e_{E},e_{W}\}$

In [141]:
def P_3(eps, S):
    '''
    Calculates: P(S), where S\subseteq\{e_{N},e_{S},e_{E},e_{W}\}
    Arguments: S a dictionary with keywords in Directions and values in
    {True,False}
               0 <= eps <= 1
    '''    
    def findModifications(mod, query, result, indices):
        if mod==0:
            result.append(query)
            return [query]
        for i in range(0,len(query)):
            if i in indices: continue
            a=str(int(not int(query[i])))
            newQ=query[:i]+a+query[i+1:]
            findModifications(mod-1, newQ, result, indices+[i])
        return list(set(result))
    
    global jointProb
    localJointProb={}
    indices={}
    if Directions.NORTH in S: indices[0]=S[Directions.NORTH]
    if Directions.SOUTH in S: indices[1]=S[Directions.SOUTH]
    if Directions.EAST in S: indices[2]=S[Directions.EAST]
    if Directions.WEST in S: indices[3]=S[Directions.WEST]
    
    for x in jointProb.values():
        for row in x:
            key=''
            for index in indices.keys():
                key=key+str(row[index])
            if key not in localJointProb: localJointProb[key]=row[4]
            else: localJointProb[key]+=row[4]
    
    query=''
    for i in range(0,4):
        if i not in indices: continue
        query=query+str(int(indices[i]))
    
    prob=0
    for mod in range (0, len(query)+1):
        for newQ in findModifications(mod, query, [], []):
            prob+=(localJointProb[newQ])*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
    
    return prob

P_3(0.3, {Directions.EAST: True, Directions.SOUTH: False})

0.24833333333333332

### c. Bayes' net for dynamic perception and position.

Now we will consider a scenario where the Pacman moves a finite number of steps $n$. In this case we have $n$
different variables for the positions $X_{1},\dots,X_{n}$, as well as for each one of the perceptions, e.g.
$E_{N_{1}},\dots,E_{N_{n}}$ for the north perception. For the initial Pacman position, assume an uniform 
distribution among the valid positions. Also assume that at each time step the Pacman choses, to move, one of the valid neighbor positions with uniform probability. Draw the corresponding Bayes' net for $n=4$.

<img src="bayesNetB.png" />

### d. Probability functions calculated from the dynamic model.

Assuming an uniform distribution for the Pacman position probability, write functions to calculate the following probabilities:

i. $P(X_{4}=x_{4}|E_{1}=e_{1},E_{3}=e_{3})$

In [142]:
def P_4(eps, E_1, E_3):
    #State Transition probability = {X_2{ X_1:P(X_2|X_1) }}
    tabuX=[(2,2),(2,4),(4,2),(4,4),(5,2),(5,4)]
    STP={(x,y):{(x+i[0],y+i[1]):0 for i in [(1,0), (0,1), (-1,0), (0, -1)] if(0<(x+i[0])<7 and 0<(y+i[1])<6 and (x+i[0],y+i[1]) not in tabuX)} for x in range(1,7) for y in range(1,6) if (x,y) not in tabuX}
    for x1 in STP.keys():
        for x2 in STP[x1].keys():
            STP[x1][x2]=1.0/len(STP[x2])
    
    '''
    Calculates: P(X_{4}=x_{4}|E_{1}=e_{1},E_{3}=e_{3})
    Arguments: E_1, E_3 dictionaries of type Directions --> {True,False}
               0 <= eps <= 1
    '''
    def findProb(x,q):
        for row in jointProb[x]:
            if row[0]==int(q[0]) and row[1]==int(q[1]) and row[2]==int(q[2]) and row[3]==int(q[3]):
                return row[4]
            
    def findModifications(mod, query, result, indices):
        if mod==0:
            result.append(query)
            return [query]
        for i in range(0,len(query)):
            if i in indices: continue
            a=str(int(not int(query[i])))
            newQ=query[:i]+a+query[i+1:]
            findModifications(mod-1, newQ, result, indices+[i])
        return list(set(result))
    
    global jointProb
    probX1={}
    jointE_1 = P_3(eps, E_1)
    query= str(int(E_1[Directions.NORTH]))+str(int(E_1[Directions.SOUTH]))+str(int(E_1[Directions.EAST]))+str(int(E_1[Directions.WEST]))
    for x in jointProb.keys():
        prob=0
        for mod in range (0, len(query)+1):
            for newQ in findModifications(mod, query, [], []):
                prob+=findProb(x,newQ)*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
        probX1[x]=prob/jointE_1
    
    probX2={}
    for x in STP.keys():
        suma=0
        for key in STP[x].keys():
            suma+=STP[x][key]*probX1[key]
        probX2[x]=suma
    
    probX3_X2= {}
    for x in STP.keys():
        suma=0
        for key in STP[x].keys():
            suma= STP[x][key]*probX2[key]
        probX3_X2[x]=suma
    
    probX3={}
    jointE_3 = P_3(eps, E_3)
    query= str(int(E_3[Directions.NORTH]))+str(int(E_3[Directions.SOUTH]))+str(int(E_3[Directions.EAST]))+str(int(E_3[Directions.WEST]))
    for x in jointProb.keys():
        prob=0
        for mod in range (0, len(query)+1):
            for newQ in findModifications(mod, query, [], []):
                prob+=findProb(x,newQ)*(eps**(0+mod))*(1-eps)**(len(newQ)-mod)
        probX3[x]=prob/jointE_3
    
    totalProbX3={}
    suma=0
    for x in probX3_X2.keys():
        totalProbX3[x]=probX3[x]*probX3_X2[x]*24.0
        suma+=totalProbX3[x]
    for x in probX3_X2.keys():
        totalProbX3[x]=totalProbX3[x]/suma
    
    probX4= {}      
    for x in STP.keys():
        suma=0
        for key in STP[x].keys():
            suma+=STP[x][key]*totalProbX3[key]
        probX4[x]=suma
    
    pd = {(x,y):0 for x in range(1,7) for y in range(1,6)}
    for k  in probX4.keys():
        pd[k]=probX4[k]
    
    return pd

E_1 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
E_3 = {Directions.NORTH: True, Directions.SOUTH: True, Directions.EAST: False, Directions.WEST: False}
P_4(0.1, E_1, E_3)

{(1, 1): 0.04264966977161866,
 (1, 2): 4.3691032902958596e-05,
 (1, 3): 0.04265616926411662,
 (1, 4): 0.00030367073282138997,
 (1, 5): 0.04264966977161866,
 (2, 1): 0.00015923756620003914,
 (2, 2): 0,
 (2, 3): 0.00021836489378565473,
 (2, 4): 0,
 (2, 5): 0.010548676324190352,
 (3, 1): 0.08529284005073937,
 (3, 2): 0.0003339114270827353,
 (3, 3): 0.19190726524103904,
 (3, 4): 0.010463370485154616,
 (3, 5): 0.08529284005073937,
 (4, 1): 0.023037451159022,
 (4, 2): 0,
 (4, 3): 0.03217979979396609,
 (4, 4): 0,
 (4, 5): 0.05290911867964976,
 (5, 1): 0.04448577640229258,
 (5, 2): 0,
 (5, 3): 0.1493875853193796,
 (5, 4): 0,
 (5, 5): 0.06422798486484846,
 (6, 1): 0.022907461309062783,
 (6, 2): 0.001979095465629059,
 (6, 3): 0.03199537669433645,
 (6, 4): 0.02172130392818494,
 (6, 5): 0.04264966977161866}

ii. $P(X_{2}=x_{2}|E_{2}=e_{2},E_{3}=e_{3},E_{4}=e_{4})$

In [143]:
def P_5(eps, E_2, E_3, E_4):
    '''
    Calculates: P(X_{2}=x_{2}|E_{2}=e_{2},E_{3}=e_{3},E_{4}=e_{4})
    Arguments: E_2, E_3, E_4 dictionaries of type Directions --> {True,False}
               0 <= eps <= 1
    '''
    pd = {(x,y):0 for x in range(1,7) for y in range(1,6)}
    return pd

E_2 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
E_3 = {Directions.NORTH: True, Directions.SOUTH: True, Directions.EAST: False, Directions.WEST: False}
E_4 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
P_5(0.1, E_2, E_3, E_4)

{(1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (1, 4): 0,
 (1, 5): 0,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (2, 4): 0,
 (2, 5): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0,
 (3, 4): 0,
 (3, 5): 0,
 (4, 1): 0,
 (4, 2): 0,
 (4, 3): 0,
 (4, 4): 0,
 (4, 5): 0,
 (5, 1): 0,
 (5, 2): 0,
 (5, 3): 0,
 (5, 4): 0,
 (5, 5): 0,
 (6, 1): 0,
 (6, 2): 0,
 (6, 3): 0,
 (6, 4): 0,
 (6, 5): 0}

iii. $P(E_{4}=e_{4}|E_{1}=e_{1},E_{2}=e_{2},E_{3}=e_{3})$

In [144]:
def P_6(eps, E_1, E_2, E_3):
    '''
    Calculates: P(E_{4}=e_{4}|E_{1}=e_{1},E_{2}=e_{2},E_{3}=e_{3})
    Arguments: E_1, E_2, E_3 dictionaries of type Directions --> {True,False}
               0 <= eps <= 1
    '''
    pd = {(n, s, e, w): 0 for n in [False, True] for s in [False, True] 
                                 for e in [False, True] for w in [False, True]}
    return pd

E_1 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
E_2 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
E_3 = {Directions.NORTH: True, Directions.SOUTH: True, Directions.EAST: False, Directions.WEST: False}
P_6(0.1, E_1, E_2, E_3)

{(False, False, False, False): 0,
 (False, False, False, True): 0,
 (False, False, True, False): 0,
 (False, False, True, True): 0,
 (False, True, False, False): 0,
 (False, True, False, True): 0,
 (False, True, True, False): 0,
 (False, True, True, True): 0,
 (True, False, False, False): 0,
 (True, False, False, True): 0,
 (True, False, True, False): 0,
 (True, False, True, True): 0,
 (True, True, False, False): 0,
 (True, True, False, True): 0,
 (True, True, True, False): 0,
 (True, True, True, True): 0}

iv. $P(E_{E_{2}}=e_{E_{2}}|E_{N_{2}}=e_{N_{2}},E_{S_{2}}=E_{S_{2}})$

In [145]:
def P_7(eps, E_N, E_S):
    '''
    Calculates: P(E_{E_{2}}=e_{E_{2}}|E_{N_{2}}=e_{N_{2}},E_{S_{2}}=E_{S_{2}})
    Arguments: E_N_2, E_S_2 \in {True,False}
               0 <= eps <= 1
    '''
    return P_2(eps, E_N, E_S)
    pd = {True:0, False:0}
    return pd

P_7(0.1, True, False)

{False: 0.615686274509804, True: 0.3843137254901961}

### Test functions

You can use the following functions to test your solutions.

In [147]:
def approx_equal(val1, val2):
    return abs(val1-val2) <= 0.00001

def test_P_1():
    pd = P_1(0.0, True, True)
    assert approx_equal(pd[(2, 1)], 0.1111111111111111)
    assert approx_equal(pd[(3, 1)], 0)
    pd = P_1(0.3, True, False)
    assert approx_equal(pd[(2, 1)], 0.03804347826086956)
    assert approx_equal(pd[(3, 1)], 0.016304347826086956)

def test_P_2():
    pd = P_2(0.0, True, True)
    assert approx_equal(pd[False], 1.0)
    pd = P_2(0.3, True, False)
    assert approx_equal(pd[False], 0.5514492753623188)

def test_P_3():
    pd = P_3(0.1, {Directions.EAST: True, Directions.WEST: True})
    assert approx_equal(pd, 0.2299999999999999)
    pd = P_3(0.1, {Directions.EAST: True})
    assert approx_equal(pd, 0.3999999999999999)
    pd = P_3(0.2, {Directions.EAST: False, Directions.WEST: True, Directions.SOUTH: True})
    assert approx_equal(pd, 0.0980000000000000)

def test_P_4():
    E_1 = {Directions.NORTH: False, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: True}
    E_3 = {Directions.NORTH: False, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: True}
    pd = P_4(0.0, E_1, E_3)
    print pd[6,3]
    print pd[4,3]
    assert approx_equal(pd[(6, 3)], 0.1842105263157895)
    assert approx_equal(pd[(4, 3)], 0.0)
    pd = P_4(0.2, E_1, E_3)
    assert approx_equal(pd[(6, 3)], 0.17777843398830864)
    assert approx_equal(pd[(4, 3)], 0.000578430282649176)
    E_1 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
    E_3 = {Directions.NORTH: False, Directions.SOUTH: False, Directions.EAST: True, Directions.WEST: False}
    pd = P_4(0.0, E_1, E_3)
    assert approx_equal(pd[(6, 2)], 0.3333333333333333)
    assert approx_equal(pd[(4, 3)], 0.0)

def test_P_5():
    E_2 = {Directions.NORTH: True, Directions.SOUTH: True, Directions.EAST: False, Directions.WEST: False}
    E_3 = {Directions.NORTH: True, Directions.SOUTH: False, Directions.EAST: False, Directions.WEST: False}
    E_4 = {Directions.NORTH: True, Directions.SOUTH: True, Directions.EAST: False, Directions.WEST: False}
    pd = P_5(0, E_2, E_3, E_4)
    assert approx_equal(pd[(2, 5)], 0.5)
    assert approx_equal(pd[(4, 3)], 0.0)
    pd = P_5(0.3, E_2, E_3, E_4)
    assert approx_equal(pd[(2, 5)], 0.1739661245168835)
    assert approx_equal(pd[(4, 3)], 0.0787991740545979)

def test_P_7():
    pd = P_7(0.0, True, False)
    print pd[False]
    assert approx_equal(pd[False], 0.7142857142857143)
    pd = P_7(0.3, False, False)
    assert approx_equal(pd[False], 0.5023529411764706)
    
test_P_1()
test_P_2()
test_P_3()
print 'Everything seems ok so far =)'

0.166666666667
0.0


AssertionError: 